# Testes da rotina que gera os vetores de entrada com base em uma série temporal

## 1. Importando as bibliotecas necessárias

### Bibliotecas obrigatórias

In [5]:
import numpy as np

### Bibliotecas não-obrigatórias

In [2]:
import sys 
sys.path.insert(0, '../../scripts')
import timeseries as ts

## 2. Definindo a função (debug)

Não é necessário executar essa linha se você estiver importando o arquivo com a classe.

In [6]:
class SerieTemporal:

    def __init__(self, dados, K, L):
        """
        Descrição:
        ----------
        Construtor da classe 'SerieTemporal'

        Parâmetros:
        -----------
        dados: np.ndarray
            Conjunto de valores da série temporal
        K: int
            O número de entradas utilizado para a predição
        L: int
            O passo de predição     
        """

        if not type(dados) is np.ndarray:
            raise TypeError("Os dados devem ser um array do numpy!")        

        if (K <= 0):
            raise ValueError("O hiperparâmetro 'K' deve ser um inteiro positivo!")

        if (L > len(dados)):
            raise ValueError("L deve ser menor que o número de dados temporais!")

        self.__dados = dados
        self.__K = K
        self.__L = L
        self._matriz_entrada = np.array([])
        self._matriz_saida = np.array([])

        pass

    def _criar_vetores(self, indice):
        """
        Descrição:
        ----------
        Função para criar os vetores de entrada e saída para as etapas de treinamento e teste para uma série temporal.
    
        Parâmetros:
        -----------
        indice: int
            A posicao a partir da qual se deseja prever algum valor
        """

        K, L = self.__K, self.__L
        dados = self.__dados
    
        # checa se a partir da posição atual podemos criar um vetor de amostras dado um K
        if ((indice + 1) < (K - 1)):
            raise ValueError("(indice + 1) = "+str(indice + 1)+" deve ser maior ou igual a (K - 1) = "+str(K - 1)+" !")
        
        # checa se o valor que queremos prever (que vai ser armazenado na matriz de saida), está dentro da série temporal
        if ((indice+L) > (len(dados)-1)):
            raise ValueError("O passo de predição (L = "+str(L)+") somado com o índice atual (indice = "+str(indice)+") não deve estourar o número de dados na série temporal!")
    
        vetor_entrada = np.array(dados[(indice-(K-1)):(indice+1)])
        vetor_entrada = np.insert(vetor_entrada, 0, 1) # insere o elemento x^0 no vetor
        vetor_saida = np.array(dados[indice+L])

        return vetor_entrada, vetor_saida    

    def dividir_treino_teste(self, tam_teste):
        """
        Descrição:
        ----------
        Função para selecionar os 'n' primeiros dados das matrizes para o treinamento.

        Parâmetros:
        -----------
        tam_teste: float
            Proporção de dados que iremos separar para o teste. Deve ser entre 0.0 e 1.0.
        """
        
        if ((tam_teste < 0.0) | (tam_teste > 1.0)):
            raise ValueError("A proporção dos dados de teste deve ser entre 0.0 e 1.0!")
            
        matriz_entrada = self._matriz_entrada
        matriz_saida = self._matriz_saida
            
        tam_treino = 1.0 - tam_teste
        n_dados = len(matriz_saida)

        return (matriz_entrada[:int(tam_treino*n_dados)], matriz_entrada[int(n_dados*(1 - tam_teste)):],
                matriz_saida[:int(tam_treino*n_dados)], matriz_saida[int(n_dados*(1 - tam_teste)):])

    def criar_matrizes(self):
        """
        Descrição:
        ----------
        Função para criar as matrizes com os vetores de entrada e saída para as etapas de treinamento e teste.
    
        Parâmetros:
        -----------
        Nenhum
        """
    
        K, L = self.__K, self.__L
        num_dados = len(self.__dados)
    
        for indice in range((K-1), (num_dados-L)):
            vetor_entrada, vetor_saida = self._criar_vetores(indice)
        
            if (len(self._matriz_entrada) == 0):
                self._matriz_entrada = vetor_entrada
            else:
                self._matriz_entrada = np.vstack((self._matriz_entrada, vetor_entrada))
                
            if (len(self._matriz_saida) == 0):
                self._matriz_saida = np.array([vetor_saida])
            else:
                self._matriz_saida = np.vstack((self._matriz_saida, vetor_saida))

        pass

## 3. Testando

In [7]:
serie_temporal = SerieTemporal(dados=np.array(range(0, 100)), K=4, L=1)

In [8]:
serie_temporal.criar_matrizes()

In [9]:
X_treino, X_teste, y_treino, y_teste = serie_temporal.dividir_treino_teste(tam_teste=0.35)

In [ ]:
print("Matriz de entrada do conjunto de treinamento:\n", X_treino, "\n")
print("Matriz de saída do conjunto de treinamento:\n", y_treino, "\n")
print("Matriz de entrada do conjunto de teste:\n", X_teste, "\n")
print("Matriz de saída do conjunto de teste:\n", y_teste)